In [2]:
import h5py
import numpy as np
import os
import imageio
import cv2
import math
import scipy
import scipy.io
import scipy.misc
import skimage
import skimage.io
import skimage.transform
from functools import lru_cache
from tqdm import tqdm as tqdm
from vectormath import Vector2

In [3]:
class H36Mparser(object):
    data = [
        'S',
        'center',
        'part',
        'scale',
        'zind',
    ]
    
    def __init__(self):
        self.annotation = h5py.File('C:/Users/I.B. Park/Desktop/h36m/annot/train.h5', 'r')
        self.image_name = np.genfromtxt('C:/Users/I.B. Park/Desktop/h36m/annot/train_images.txt', dtype = str)
        self.shuffle = np.arange(len(self))
    
    @lru_cache(maxsize=1)
    def __len__(self):
        return len(self.image_name)
    
    def __iter__(self):
        self.index = 0
        np.random.shuffle(self.shuffle)
        return self
    
    def __next__(self):
        if self.index >= len(self):
            raise StopIteration
        index = self.shuffle[self.index]
        self.index = self.index + 1
        return [self.annotation[tag][index] for tag in H36Mparser.data] + [self.image_name[index]]

In [4]:
@lru_cache(maxsize=32)
def gaussian(size, sigma=0.25, mean=0.5):
    width = size
    heigth = size
    amplitude = 1.0
    sigma_u = sigma
    sigma_v = sigma
    mean_u = mean * width + 0.5
    mean_v = mean * heigth + 0.5

    over_sigma_u = 1.0 / (sigma_u * width)
    over_sigma_v = 1.0 / (sigma_v * heigth)

    x = np.arange(0, width, 1, np.float32)
    y = x[:, np.newaxis]

    du = (x + 1 - mean_u) * over_sigma_u
    dv = (y + 1 - mean_v) * over_sigma_v

    return amplitude * np.exp(-0.5 * (du * du + dv * dv))


def generateHeatmap(size, y0, x0, sigma=1):
    pad = 3 * sigma
    y0, x0 = int(y0), int(x0)
    dst = [max(0, y0 - pad), max(0, min(size, y0 + pad + 1)), max(0, x0 - pad), max(0, min(size, x0 + pad + 1))]
    src = [-min(0, y0 - pad), pad + min(pad, size - y0 - 1) + 1, -min(0, x0 - pad), pad + min(pad, size - x0 - 1) + 1]

    heatmap = np.zeros([size, size])
    g = gaussian(3 * 2 * sigma + 1)
    heatmap[dst[0]:dst[1], dst[2]:dst[3]] = g[src[0]:src[1], src[2]:src[3]]

    return heatmap


def cropImage(image, center, scale, rotate, resolution):
    center = Vector2(center)  # assign new array
    height, width, _ = image.shape
    crop_ratio = 200 * scale / resolution
    if crop_ratio >= 2:  # if box size is greater than two time of resolution px
        # scale down image
        height = math.floor(height / crop_ratio)
        width = math.floor(width / crop_ratio)

        if max([height, width]) < 2:
            # Zoomed out so much that the image is now a single pixel or less
            raise ValueError("Width or height is invalid!")

        image = skimage.transform.resize(image, (height, width), mode='constant')
        center /= crop_ratio
        scale /= crop_ratio

    ul = (center - 200 * scale / 2).astype(int)
    br = (center + 200 * scale / 2).astype(int)  # Vector2

    if crop_ratio >= 2:  # force image size 256 x 256
        br -= (br - ul - resolution)

    pad_length = math.ceil((ul - br).length - (br.x - ul.x) / 2)

    if rotate != 0:
        ul -= pad_length
        br += pad_length

    src = [max(0, ul.y), min(height, br.y), max(0, ul.x), min(width, br.x)]
    dst = [max(0, -ul.y), min(height, br.y) - ul.y, max(0, -ul.x), min(width, br.x) - ul.x]

    new_image = np.zeros([br.y - ul.y, br.x - ul.x, 3], dtype=np.float64)
    new_image[dst[0]:dst[1], dst[2]:dst[3], :] = image[src[0]:src[1], src[2]:src[3], :]

    if rotate != 0:
        new_image = skimage.transform.rotate(new_image, rotate)
        new_height, new_width, _ = new_image.shape
        new_image = new_image[pad_length:new_height - pad_length, pad_length:new_width - pad_length, :]

    if crop_ratio < 2:
        new_image = skimage.transform.resize(new_image, (resolution, resolution), mode='constant')

    return new_image

In [5]:
def generateVoxel(voxel_xy_resolution, voxel_z_resolution, xy, z, heatmap_xy_coefficient, heatmap_z_coefficient):
    volume = np.zeros(shape = (voxel_z_resolution, voxel_xy_resolution, voxel_xy_resolution), dtype = np.float64)
    xy_view = generateHeatmap(size = voxel_xy_resolution, y0 = xy[1], x0 = xy[0], sigma = heatmap_xy_coefficient)
    z_view = gaussian(heatmap_z_coefficient)[math.ceil(heatmap_z_coefficient/2)]
    cnt = 0
    for i in range(z - math.floor(heatmap_z_coefficient/2), z + math.floor(heatmap_z_coefficient/2)):
        if 0 <= i < voxel_z_resolution:
            volume[i] = z_view[cnt] * xy_view
        cnt = cnt + 1
    return volume

In [6]:
def decode_image_name(image_name):
    subject_action, camera_frame, _ = image_name.split('.')
    split = subject_action.split('_')
    subject = split[0]
    action = split[1]
    if len(split) >= 3:
        action = action + '_' + split[2]
    camera, frame = camera_frame.split('_')
    
    return subject, action, camera, frame

In [7]:
parser = H36Mparser()

z_limits = np.squeeze(scipy.io.loadmat('C:/Users/I.B. Park/Desktop/c2f-vol-demo-master/matlab/utils/data/voxel_limits.mat')['limits'])
z_centers = (z_limits[1:65] + z_limits[0:64]) / 2
z_delta = z_limits[32]

In [40]:
data_dir = 'D:/data/Human3.6M/converted'

# Voxel heatmap sigma.
heatmap_xy_coefficient = 2
heatmap_z_coefficient = -1

# Voxel resolution.
voxel_xy_resolution = 64
voxel_z_fine_resolution = 64
voxel_z_coarse_resolution = 64

with tqdm(range(len(parser))) as progress:
    for S, center, part, scale, zind, image_name in parser:
        
        # Extract subject and camera name from an image name.
        subject, _, camera, _ = decode_image_name(image_name)

        # Pre-calculate constants.
        # heatmap_z_coefficient is 1, 1, 1, 3, 5, 7, 13 for 1, 2, 4, 8, 16, 32, 64.
        heatmap_z_coefficient = 2 * math.floor((6 * heatmap_xy_coefficient * voxel_z_coarse_resolution / voxel_z_fine_resolution + 1)/2) + 1
        image_xy_resolution = 200 * scale
        
        # Crop RGB image.
        image = skimage.io.imread('%s/%s-2/%s' % (data_dir, subject, image_name))
        image = cropImage(image, center, scale, 0, 256)

        # Convert the coordinate from a RGB image to a cropped RGB image.
        xy = voxel_xy_resolution * (part - center) / image_xy_resolution + voxel_xy_resolution * 0.5

        # Build voxel.
        voxel = np.zeros(shape = (len(part) * voxel_z_coarse_resolution, voxel_xy_resolution, voxel_xy_resolution))
        for part_idx in range(len(part)):
            # zind range (1, 64)
            # z range (0, 63)
            z = math.ceil(zind[part_idx] * voxel_z_coarse_resolution / voxel_z_fine_resolution)
            voxel[part_idx * voxel_z_coarse_resolution : (part_idx + 1) * voxel_z_coarse_resolution] = generateVoxel(
                voxel_xy_resolution, voxel_z_coarse_resolution,
                xy[part_idx], z,
                heatmap_xy_coefficient, heatmap_z_coefficient)
            
        '''
        z_root = S[0, 2] + z_delta
        z_relative = z_centers[zind - 1]
        z = z_root + z_relative

        f = np.loadtxt('calibration/%s_f.txt' % camera)
        c = np.loadtxt('calibration/%s_c.txt' % camera)
        k = np.loadtxt('calibration/%s_k.txt' % camera)
        p = np.loadtxt('calibration/%s_p.txt' % camera)

        x = (part[:, 0] - c[0]) * z / f[0]
        y = (part[:, 1] - c[1]) * z / f[1]

        reconstructed = np.stack([x, y, z]).transpose(1, 0)
        for distance in reconstructed - S:
            distance = np.linalg.norm(distance)

            avg_error = avg_error + distance / (len(parser) * len(part))

        '''
        '''
        xy_heatmap = np.zeros(shape = (part.shape[0], voxel_xy_resolution, voxel_xy_resolution))

        for joint_idx in range(part.shape[0]):
            xy_heatmap[joint_idx] = generateHeatmap(voxel_xy_resolution, xy[joint_idx][1], xy[joint_idx][0], heatmap_xy_coefficient)
        '''
        
        
        for z in range(voxel_z_coarse_resolution):
            for y in range(voxel_xy_resolution):
                for x in range(voxel_xy_resolution):
                    voxel[z, y, x] = np.max(voxel[[part * voxel_z_coarse_resolution + z for part in range(len(part))], y, x])

        # image = skimage.io.imread('%s/%s-2/%s' % (data_dir, subject, image_name))
        # image = cropImage(image, center, scale, 0, 256)
        imageio.imwrite('test.png', image)
        for z in range(voxel_z_coarse_resolution):
            imageio.imwrite('heatmap%02d.png' % z, voxel[z, :, :])
        
        progress.update(1)
        break

  0%|                                                                                                            | 0/312188 [00:00<?, ?it/s]WARNING:root:Lossy conversion from float64 to uint8. Range [0.9546279907203753, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


  0%|                                                                                                | 1/312188 [00:05<460:49:03,  5.31s/it]


In [32]:
print(np.min(parser.annotation['part'][:, :, 0]), np.min(parser.annotation['part'][:, :, 1]))
print(np.max(parser.annotation['part'][:, :, 0]), np.max(parser.annotation['part'][:, :, 1]))
print(np.argmax(parser.annotation['part'][:, :, 0], 0), np.argmax(parser.annotation['part'][:, :, 0], 1))

for index in np.argmax(parser.annotation['part'][:, :, 0], 0):
    part = np.argmax(parser.annotation['part'][index, :, 0], 0)
    image_name = parser.image_name[index]
    subject, _, camera, _ = decode_image_name(image_name)
    print(index, part, parser.annotation['part'][index, part, :], image_name)
    image = skimage.io.imread('%s/%s-2/%s' % (data_dir, subject, image_name))
    imageio.imwrite('test.png', image)

26.6379737854 65.2688064575
1069.51525879 872.157531738
[ 58706  58706  58706  58711 309030 309031  58708 309031 309030 311876
 275734 309031 162193  72986 309030 309030  40585] [15 15 15 ..., 13 13 13]
58706 6 [ 946.85723877  548.83477783] S5_Discussion_2.58860488_004511.jpg
58706 6 [ 946.85723877  548.83477783] S5_Discussion_2.58860488_004511.jpg
58706 6 [ 946.85723877  548.83477783] S5_Discussion_2.58860488_004511.jpg
58711 6 [ 941.17199707  541.5166626 ] S5_Discussion_2.58860488_004536.jpg
309030 15 [ 1007.0904541    368.27191162] S8_Walking_1.60457274_002731.jpg
309031 15 [ 1005.95770264   364.03140259] S8_Walking_1.60457274_002736.jpg
58708 6 [ 947.03924561  547.58380127] S5_Discussion_2.58860488_004521.jpg
309031 15 [ 1005.95770264   364.03140259] S8_Walking_1.60457274_002736.jpg
309030 15 [ 1007.0904541    368.27191162] S8_Walking_1.60457274_002731.jpg
311876 15 [ 964.93914795  399.12380981] S8_Walking.60457274_002136.jpg
275734 16 [ 1023.88323975   356.5803833 ] S8_Phoning_1.6

In [ ]:
z_limits = np.squeeze(scipy.io.loadmat('C:/Users/I.B. Park/Desktop/c2f-vol-demo-master/matlab/utils/data/voxel_limits.mat')['limits'])
z_centers = (z_limits[1:65] + z_limits[0:64]) / 2
z_delta = z_limits[32]

In [ ]:
annotation = h5py.File('C:/Users/I.B. Park/Desktop/h36m/annot/train.h5', 'r')
# annotation = h5py.File('C:/Users/I.B. Park/Desktop/c2f-vol-demo-master/data/h36m-sample/annot/valid.h5', 'r')
image_name = np.genfromtxt('C:/Users/I.B. Park/Desktop/h36m/annot/train_images.txt', dtype = str)

In [ ]:
image_idx = 158213 # 981

In [ ]:
print('image:', image_name[image_idx])
print('camera:', image_name[image_idx].split('.')[1].split('_')[0])
for key, value in annotation.items():
    print(key, type(value[image_idx]))
    print(value[image_idx])
    if type(value[image_idx]) is np.ndarray:
        print(value[image_idx].shape)
image = imageio.imread(os.path.join('C:/Users/I.B. Park/Desktop/S1-2', image_name[image_idx]))
imageio.imwrite('test.png', image)

In [ ]:
x = annotation['part'][image_idx][:, 0]
y = annotation['part'][image_idx][:, 1]
print((np.max(x) + np.min(x))/2)
print((np.max(y) + np.min(y))/2)

In [ ]:
max_length = -1
with tqdm(range(annotation['S'].shape[0])) as progress:
    for S in annotation['S']:
        '''
        for keypoint in S:
            delta = keypoint - S[0]
            length = np.linalg.norm(delta)
            if max_length < length:
                max_length = length
        '''
        x, y, z = [S[:, 0], S[:, 1], S[:, 2]]
        delta_x = np.max(x) - np.min(x)
        delta_y = np.max(y) - np.min(y)
        delta_z = np.max(z) - np.min(z)

        length = np.linalg.norm([delta_x, delta_y, delta_z])
        if max_length < length:
            max_length = length
            
        progress.update(1)
print(max_length)

In [ ]:
# image_idx = 158213
for image_idx in tqdm(range(annotation['S'].shape[0])):
    z = annotation['S'][image_idx][:, -1]
    z_center = (np.max(z) + np.min(z)) / 2
    z_center = z[0]
    z_index = (z - z_center + 12.163020574832672)/(31.0319651) + 33
    z_index = np.floor(z_index).astype(int)
    hit = ((z_index == annotation['zind'][image_idx]) == True).all()
    if not hit:
        print(image_idx)
        print(z_index)
        break

In [ ]:
print((annotation['zind'][:][0] == 33).all())
print(annotation['zind'][:20][0])

In [ ]:
c = annotation['center'][image_idx]
pelvis = c #annotation['part'][image_idx][0]
h = 200.0 * annotation['scale'][image_idx]
res = 64.0
converted = pelvis * res / h + res * (-c / h + 0.5)
print(converted)

In [ ]:
# for part_idx in tqdm(range(len(annotation['part'][image_idx]))):
# x, y = annotation['part'][image_idx][part_idx].astype(int)
x, y = annotation['part'][image_idx][0].astype(int)
for tx in range(-5, 5):
    for ty in range(-5, 5):
        xx = x + tx
        yy = y + ty
        if not 0 <= xx < image.shape[1] or not 0 <= yy < image.shape[0]:
            continue
        image[yy, xx, :] = [255, 0, 0]
imageio.imwrite('test.png', image)

In [ ]:
t = int(100 * annotation['scale'][image_idx])
for tx in range(-t, t):
    for ty in range(-t, t):
        xx = x + tx
        yy = y + ty
        if not 0 <= xx < image.shape[1] or not 0 <= yy < image.shape[0]:
            continue
        image[yy, xx, :] = [255, 0, 0]
imageio.imwrite('test.png', image)

In [ ]:
mine = [
    [30.776466, -359.742493, 5252.067383],
    [-67.319511, -339.605377, 5164.619629],
    [-52.176018, 94.260391, 5252.292969],
    [-9.488297, 545.306580, 5284.522461],
    [128.872269, -379.879578, 5339.514648],
    [112.328552, 61.875256, 5366.618164],
    [132.056335, 515.574890, 5375.042480],
    [87.822304, -573.780762, 5178.282715],
    [93.088982, -821.356262, 5109.235352],
    [60.073814, -909.662292, 5185.299316],
    [78.938721, -1011.616516, 5135.550293],
    [-23.573996, -758.941406, 5036.396484],
    [-216.210449, -599.674622, 5160.093262],
    [-324.774109, -706.496399, 5360.524902],
    [197.457428, -796.293518, 5215.487793],
    [198.218262, -681.528198, 5469.671875],
    [10.382108, -717.354492, 5633.383301],
]

In [ ]:
for idx in range(len(mine)):
    print(np.linalg.norm(mine[idx] - annotation['S'][image_idx][idx]))

In [ ]:
cam = image_name[image_idx].split('.')[1].split('_')[0]

f = np.loadtxt('calibration/%s_f.txt' % cam)
c = np.loadtxt('calibration/%s_c.txt' % cam)
k = np.loadtxt('calibration/%s_k.txt' % cam)
p = np.loadtxt('calibration/%s_p.txt' % cam)

distort = np.asarray([k[0], k[1], p[0], p[1], k[2]])
cam_mat = np.mat('%f 0 %f; 0 %f %f; 0 0 1' % (f[0], c[0], f[1], c[1]))

h,  w = image.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(cam_mat,distort,(w,h),1,(w,h))
map_x, map_y = cv2.initUndistortRectifyMap(cam_mat, distort, None, newcameramtx,(w,h),5)

In [ ]:
inverse = np.linalg.inv(cam_mat)
S = np.ndarray(shape=(len(mine), 3), dtype=np.float)
for idx in range(len(mine)):
    y, x = annotation['part'][image_idx][idx].astype(int)
    yx = [y, x] -  annotation['part'][image_idx][0].astype(int) + 200 * annotation['scale'][image_idx]
    yx = yx / 2 * 200 * annotation['scale'][image_idx]
    S[idx] = np.squeeze(inverse * np.asarray([[yx[1], yx[0], 1.0]]).transpose((1, 0)))
print(S)

In [ ]:
z = annotation['zind'][image_idx].astype(float)
z = z - z[0]
z = 100 * 10 * z / 64
print(z)

In [ ]:
converted = (S)
converted = converted * np.expand_dims(annotation['S'][image_idx][0][-1].astype(float) + z, -1)
print(converted - converted[0])